In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [3]:
x_train, y_train,x_test_original, y_test_original = ReadTS('SwedishLeaf')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
#print(y_train)
#print(x_train[1])

(500, 128, 1)


In [4]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train[1])

In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 15)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-12-02 18:06:35.798352: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


13/13 [==============================] - 0s 15ms/step - loss: 2.5841 - sparse_categorical_accuracy: 0.1325 - val_loss: 2.6885 - val_sparse_categorical_accuracy: 0.1000
Epoch 2/100
13/13 [==============================] - 0s 9ms/step - loss: 2.2517 - sparse_categorical_accuracy: 0.2900 - val_loss: 2.6685 - val_sparse_categorical_accuracy: 0.0500
Epoch 3/100
13/13 [==============================] - 0s 9ms/step - loss: 2.1012 - sparse_categorical_accuracy: 0.3975 - val_loss: 2.6536 - val_sparse_categorical_accuracy: 0.0700
Epoch 4/100
13/13 [==============================] - 0s 8ms/step - loss: 1.9821 - sparse_categorical_accuracy: 0.4600 - val_loss: 2.6683 - val_sparse_categorical_accuracy: 0.0800
Epoch 5/100
13/13 [==============================] - 0s 8ms/step - loss: 1.8994 - sparse_categorical_accuracy: 0.5300 - val_loss: 2.7114 - val_sparse_categorical_accuracy: 0.0600
Epoch 6/100
13/13 [==============================] - 0s 8ms/step - loss: 1.7951 - sparse_categorical_accuracy: 0.575

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

20/20 [==============================] - 0s 3ms/step - loss: 0.5888 - sparse_categorical_accuracy: 0.7984
Test accuracy 0.7983999848365784
Test loss 0.5887854099273682


In [7]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    print(average_l2_norm)

    return average_l2_norm


def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [9]:
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 0
index_1 = 30

strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'SwedishLeaf.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'SwedishLeafl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

20/20 [==============================] - 0s 2ms/step
4.496358810528748
20/20 [==============================] - 0s 2ms/step
5.065455054305391
20/20 [==============================] - 0s 2ms/step
4.815435298365085
 1/20 [>.............................] - ETA: 0s

/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

20/20 [==============================] - 0s 2ms/step
7.807475304435886
20/20 [==============================] - 0s 2ms/step
0.6780120942945111
20/20 [==============================] - 0s 2ms/step
done
   Strategy  Accuracy  Precision  Recall  F1 Score
0  original    0.7984   0.837025  0.7984  0.784815
1       RBP    0.5296   0.648496  0.5296  0.516135
2      MRBP    0.3568   0.543904  0.3568  0.283406
3      zero    0.5056   0.614126  0.5056  0.484772
4     noise    0.3264   0.367519  0.3264  0.255350
5      blur    0.7472   0.806076  0.7472  0.730719
    L2-Norm
0  4.496359
1  5.065455
2  4.815435
3  7.807475
4  0.678012


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
